In [1]:
import os
import pandas as pd
from sklearn.preprocessing import scale, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
import numpy as np
from tensorflow import keras
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import regularizers
from keras.optimizers import Adam

# Đọc dữ liệu từ file CSV
df = pd.read_csv('dataC3_full_3_label_both_imagine_real.csv')
# df = df[df.iloc[:,-1]!=0]
# Với cột cuối là nhãn
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

EDA

In [3]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(df, title="Pandas Profiling Report for EEG Motor Imagery")
# profile.to_file("Pandas Profiling Report for EEG Motor Imagery.html")

In [4]:
import scipy as sp
def FeatureExtract(data, fs):
    f, t, Zxx = sp.signal.stft(data, fs, nperseg=len(data)/fs * fs, noverlap=len(data)/fs * fs-1)
    fig, ax2 = plt.subplots(figsize=(3,3))
    ax2.pcolormesh(t, f, np.abs(Zxx), vmin=-1, vmax=5, shading='auto')
    ax2.axis('off')
    ax2.set_ylim(0.5, 40)
    return

Feature Visualization

In [5]:
import scipy as sp
from tqdm import tqdm
# print(train.iloc[1,:].values)
fs=160
# len(train)
feature_dir0 = 'feature_STFT/0'
feature_dir1 = 'feature_STFT/1'
feature_dir2 = 'feature_STFT/2'
os.makedirs(feature_dir0, exist_ok=True)
os.makedirs(feature_dir1, exist_ok=True)
os.makedirs(feature_dir2, exist_ok=True)
for i in tqdm(range(len(X))):
    data = X[i,:]
    if y[i] == 0 and not(os.path.exists(f'{feature_dir0}/STFT{i}.png')):
        # print(i,y[i])
        f, t, Zxx = sp.signal.stft(data, fs, nperseg=len(data)/fs * fs, noverlap=len(data)/fs * fs-1)
        fig, ax2 = plt.subplots(figsize=(3,3))
        ax2.pcolormesh(t, f, np.abs(Zxx), vmin=-1, vmax=5, shading='auto')
        ax2.axis('off')
        ax2.set_ylim(0.5, 40)
        plt.savefig(f'{feature_dir0}/STFT{i}.png', dpi=100, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        
    if y[i] == 1 and not(os.path.exists(f'{feature_dir1}/STFT{i}.png')):
        # print(i,y[i])
        f, t, Zxx = sp.signal.stft(data, fs, nperseg=len(data)/fs * fs, noverlap=len(data)/fs * fs-1)
        fig, ax2 = plt.subplots(figsize=(3,3))
        ax2.pcolormesh(t, f, np.abs(Zxx), vmin=-1, vmax=5, shading='auto')
        ax2.axis('off')
        ax2.set_ylim(0.5, 40)
        plt.savefig(f'{feature_dir1}/STFT{i}.png', dpi=100, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        
    if y[i] == 2 and not(os.path.exists(f'{feature_dir2}/STFT{i}.png')):
        # print(i,y[i])
        f, t, Zxx = sp.signal.stft(data, fs, nperseg=len(data)/fs * fs, noverlap=len(data)/fs * fs-1)
        fig, ax2 = plt.subplots(figsize=(3,3))
        ax2.pcolormesh(t, f, np.abs(Zxx), vmin=-1, vmax=5, shading='auto')
        ax2.axis('off')
        ax2.set_ylim(0.5, 40)
        plt.savefig(f'{feature_dir2}/STFT{i}.png', dpi=100, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

100%|██████████| 13578/13578 [00:00<00:00, 90535.05it/s]


EEG Raw Visualization

In [3]:
import scipy as sp
from tqdm import tqdm
# print(train.iloc[1,:].values)
fs=160
# len(train)
feature_dir0 = 'Data_EEG/0'
feature_dir1 = 'Data_EEG/1'
feature_dir2 = 'Data_EEG/2'
os.makedirs(feature_dir0, exist_ok=True)
os.makedirs(feature_dir1, exist_ok=True)
os.makedirs(feature_dir2, exist_ok=True)
for i in tqdm(range(len(X)),ncols=200):
    data = X[i,:]
    if y[i] == 0 and not(os.path.exists(f'{feature_dir0}/raw{i}.png')):
        # print(i,y[i])
        fig, ax2 = plt.subplots(figsize=(8,6))
        ax2.plot(data)
        ax2.axis('off')
        plt.savefig(f'{feature_dir0}/raw{i}.png', dpi=100, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        
    if y[i] == 1 and not(os.path.exists(f'{feature_dir1}/raw{i}.png')):
        # print(i,y[i])
        fig, ax2 = plt.subplots(figsize=(8,6))
        ax2.plot(data, color='orange')
        ax2.axis('off')
        plt.savefig(f'{feature_dir1}/raw{i}.png', dpi=100, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        
    if y[i] == 2 and not(os.path.exists(f'{feature_dir2}/raw{i}.png')):
        # print(i,y[i])
        fig, ax2 = plt.subplots(figsize=(8,6))
        ax2.plot(data, color='green')
        ax2.axis('off')
        plt.savefig(f'{feature_dir2}/raw{i}.png', dpi=100, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13578/13578 [07:57<00:00, 28.45it/s]


In [71]:
# Chuẩn hóa dữ liệu
X = scale(X,axis=1)

# One-hot encoding cho nhãn
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Tạo thư mục để lưu các plot và mô hình
model_dir = 'models_Chuan'
os.makedirs(model_dir, exist_ok=True)

# Danh sách để lưu tên file của các mô hình đã train
model_filenames = []

# Tạo DataFrame để lưu history của loss và accuracy
history_df = pd.DataFrame(columns=['fold', 'epoch', 'train_loss', 'test_loss', 'train_accuracy', 'test_accuracy'])
tf.keras.backend.clear_session()

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = np.array(X_train).reshape(len(X_train), 1, X_train.shape[1])
X_test = np.array(X_test).reshape(len(X_test), 1, X_test.shape[1])
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train.shape

(19006, 1, 656)

In [75]:
# Chia thành tập huấn luyện và tập kiểm tra
time_steps = 1
n_features = X.shape[1]
# Xây dựng mô hình Bi-LSTM
model = Sequential()
model.add(Bidirectional(LSTM(units=16, activation='tanh'), input_shape=(time_steps, n_features)))
# model.add(Bidirectional(LSTM(units=16, activation='relu'), input_shape=(time_steps, n_features)))
print(model.output_shape)
model.add(Dense(units=6, activation='tanh'))
# model.add(Dropout(0.5)) #0.2
model.add(Dense(units=3, activation='softmax'))
#learning rate
lr = 0.0001
optimizer = Adam(learning_rate=lr)

# Biên dịch mô hình
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()
# Đặt callbacks để dừng sau 10 epochs
# early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, verbose=1, mode='max')

# Huấn luyện mô hình và lưu history
history = model.fit(X_train, y_train, epochs=2000, batch_size=4096, validation_data=(X_test, y_test))
# history = model.fit(X_train, y_train, epochs=2000, batch_size=2048, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Lưu mô hình
model_filename = f"BiLSTM_Brainwave_accuracy_{history.history['val_accuracy'][-1]*100:.2f}.h5"
model.save(os.path.join(model_dir, model_filename))
model_filenames.append(model_filename)

# Đánh giá mô hình trên tập kiểm tra
_, accuracy = model.evaluate(X_test, y_test)

# Vẽ đồ thị loss và accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.savefig(f"{model_dir}/{os.path.splitext(model_filename)[0]}_loss.png")
plt.clf()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.savefig(f"{model_dir}/{os.path.splitext(model_filename)[0]}_accuracy.png")
plt.clf()

# Lưu accuracy và loss của fold hiện tại vào DataFrame
for epoch, (train_loss, test_loss, train_accuracy, test_accuracy) in enumerate(zip(
    history.history['loss'],
    history.history['val_loss'],
    history.history['accuracy'],
    history.history['val_accuracy']
), 1):
    history_df = pd.concat([history_df, pd.DataFrame({
        'epoch': [epoch],
        'train_loss': [train_loss],
        'test_loss': [test_loss],
        'train_accuracy': [train_accuracy],
        'test_accuracy': [test_accuracy]
    })], ignore_index=True)

# Lưu thông tin của fold hiện tại vào file txt
with open(f'{model_dir}/{os.path.splitext(model_filename)[0]}.txt', 'w') as file:
    file.write(f"Train Loss: {history.history['loss'][-1]}\n")
    file.write(f"Train Accuracy: {history.history['accuracy'][-1]}\n")
    file.write(f"Validation Loss: {history.history['val_loss'][-1]}\n")
    file.write(f"Validation Accuracy: {history.history['val_accuracy'][-1]}\n")
tf.keras.backend.clear_session()

# Lưu DataFrame vào file CSV
history_df.to_csv('history_bi_lstm.csv', index=False)


(None, 32)
Epoch 1/2000
10/10 [==============================] - 2s 69ms/step - loss: 1.1045 - accuracy: 0.3669 - val_loss: 1.0874 - val_accuracy: 0.3941
Epoch 2/2000
10/10 [==============================] - 0s 24ms/step - loss: 1.0859 - accuracy: 0.3957 - val_loss: 1.0755 - val_accuracy: 0.4138
Epoch 3/2000
10/10 [==============================] - 0s 18ms/step - loss: 1.0741 - accuracy: 0.4106 - val_loss: 1.0674 - val_accuracy: 0.4247
Epoch 4/2000
10/10 [==============================] - 0s 14ms/step - loss: 1.0650 - accuracy: 0.4234 - val_loss: 1.0611 - val_accuracy: 0.4308
Epoch 5/2000
10/10 [==============================] - 0s 15ms/step - loss: 1.0578 - accuracy: 0.4335 - val_loss: 1.0561 - val_accuracy: 0.4365
Epoch 6/2000
10/10 [==============================] - 0s 14ms/step - loss: 1.0517 - accuracy: 0.4424 - val_loss: 1.0523 - val_accuracy: 0.4438
Epoch 7/2000
10/10 [==============================] - 0s 14ms/step - loss: 1.0464 - accuracy: 0.4511 - val_loss: 1.0490 - val_accur

C:\Users\Pc\AppData\Local\Temp\ipykernel_21216\1498996823.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  history_df = pd.concat([history_df, pd.DataFrame({


<Figure size 640x480 with 0 Axes>